## Neural network training 
### Hazard level: continuous
### Location: general, EDP-based

In [2]:
import numpy as np

import matplotlib
# matplotlib.use('TkAgg')
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, explained_variance_score,r2_score
import pickle
import time
import copy
import random


In [3]:
# Settings
pTrain = 0.8
pValidate = 0.1
pTest = 0.1

#Load data from file
df_random = pd.read_csv('random_noncollapse.csv')
df_edge = pd.read_csv('hand_selected_noncollapse.csv')

flag = 2 # 1 trained on all data, 2 on random data, and 3 on edge cases

if flag == 1:
        data = np.concatenate((df_random.to_numpy(),df_edge.to_numpy()))
if flag == 2:
        data = df_random.to_numpy()
        
# Definition of inputs and outputs
np.random.shuffle(data)

size_data = data.shape

input_data = data[:,:(size_data[1]-1)]
output_data = data[:,-1]

edge_data = df_edge.to_numpy()
edge_data_input = edge_data[:,:(size_data[1]-1)]
edge_data_label = edge_data[:,-1]

non_edge_data = df_random.to_numpy()
non_edge_data_input = non_edge_data[:,:(size_data[1]-1)]
non_edge_data_label = non_edge_data[:,-1]

# Normal standarization of data 
scaler = StandardScaler()
scaler.fit(input_data)

StandardScaler()

In [4]:
# Store the "transform" (mus, sigmas) in a pickle, to be applied to the data later
with open('scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)

input_data = scaler.transform(input_data)
edge_data_input = scaler.transform(edge_data_input)
non_edge_data_input = scaler.transform(non_edge_data_input)

# Split into training, validation and testing set
# p1 and p2 are thresholds to which the data is divided
# DATA MUST BE SHUFFLED
p1 = int(size_data[0]*pTrain) # train
train_data = input_data[:p1,:]
train_label = output_data[:p1]

p2 = int(size_data[0]*(pTrain+pValidate)) # validate and test
valid_data = input_data[p1:p2,:]
valid_label = output_data[p1:p2]
test_data = input_data[p2:,:]
test_label = output_data[p2:]

r2_train = []
r2_test = []
r2_edge = []
r2_non_edge = []

In [5]:
layer_accuracy = pd.DataFrame()
counter = 0

## Train model

In [6]:
for i in range(1,6):
    for j in range(1,16):
        # Training model. We're using scikit-learn since it works well with Numpy
        # Hyperparameters that we found are entered.
        layer = i
        units = 10*j
        mlp = MLPRegressor(hidden_layer_sizes=layer*(units,), learning_rate_init=0.0003, alpha=0.0001)
        mlp.fit(train_data, train_label)
        print('Model Fitted')

        #Saving model for posteriority
        #
        filename = "model_fitted_all_imp1_" + str(layer) + "_" + str(units) + ".pkl"
        with open(filename,'wb') as f:
            pickle.dump(mlp, f)


        #Load saved model so we don't need to train everytime

        mlp = pickle.load(open(filename,'rb'))

        # Computing values predicted by the neural network

        nn_train_label = mlp.predict(train_data)
        nn_test_label = mlp.predict(test_data)

        nn_edge_label = mlp.predict(edge_data_input)
        nn_non_edge_label = mlp.predict(non_edge_data_input)
        
        
        #Computing R2 error
        r2_train.append(r2_score(train_label,nn_train_label))
        r2_test.append(r2_score(test_label,nn_test_label))
        r2_edge.append(r2_score(edge_data_label,nn_edge_label))
        r2_non_edge.append(r2_score(non_edge_data_label,nn_non_edge_label))
        #plotting scatter plot of NN vs Label
        
        layer_accuracy.at[counter,'Layer'] = layer
        layer_accuracy.at[counter,'Units'] = units
        layer_accuracy.at[counter,'R2 train'] = r2_score(train_label,nn_train_label)
        layer_accuracy.at[counter,'R2 test'] = r2_score(test_label,nn_test_label)
        layer_accuracy.at[counter,'R2 edge'] = r2_score(edge_data_label,nn_edge_label)
        layer_accuracy.at[counter,'R2 non-edge'] = r2_score(non_edge_data_label,nn_non_edge_label)
        counter = counter + 1
        print(layer_accuracy)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model Fitted
   Layer  Units  R2 train   R2 test  R2 edge  R2 non-edge
0    1.0   10.0   0.97652  0.976031  0.98995      0.97642


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model Fitted
   Layer  Units  R2 train   R2 test   R2 edge  R2 non-edge
0    1.0   10.0  0.976520  0.976031  0.989950     0.976420
1    1.0   20.0  0.984302  0.983743  0.987344     0.984192


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

Model Fitted
    Layer  Units  R2 train   R2 test   R2 edge  R2 non-edge
0     1.0   10.0  0.976520  0.976031  0.989950     0.976420
1     1.0   20.0  0.984302  0.983743  0.987344     0.984192
2     1.0   30.0  0.986497  0.985423  0.990265     0.986317
3     1.0   40.0  0.987976  0.987165  0.993334     0.987795
4     1.0   50.0  0.989496  0.988592  0.995114     0.989317
..    ...    ...       ...       ...       ...          ...
68    5.0   90.0  0.998842  0.997908  0.998930     0.998654
69    5.0  100.0  0.999100  0.998060  0.998652     0.998887
70    5.0  110.0  0.999018  0.997529  0.998769     0.998731
71    5.0  120.0  0.999177  0.997657  0.998451     0.998872
72    5.0  130.0  0.999315  0.997716  0.998228     0.998998

[73 rows x 6 columns]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model Fitted
    Layer  Units  R2 train   R2 test   R2 edge  R2 non-edge
0     1.0   10.0  0.976520  0.976031  0.989950     0.976420
1     1.0   20.0  0.984302  0.983743  0.987344     0.984192
2     1.0   30.0  0.986497  0.985423  0.990265     0.986317
3     1.0   40.0  0.987976  0.987165  0.993334     0.987795
4     1.0   50.0  0.989496  0.988592  0.995114     0.989317
..    ...    ...       ...       ...       ...          ...
69    5.0  100.0  0.999100  0.998060  0.998652     0.998887
70    5.0  110.0  0.999018  0.997529  0.998769     0.998731
71    5.0  120.0  0.999177  0.997657  0.998451     0.998872
72    5.0  130.0  0.999315  0.997716  0.998228     0.998998
73    5.0  140.0  0.999219  0.997036  0.998470     0.998772

[74 rows x 6 columns]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model Fitted
    Layer  Units  R2 train   R2 test   R2 edge  R2 non-edge
0     1.0   10.0  0.976520  0.976031  0.989950     0.976420
1     1.0   20.0  0.984302  0.983743  0.987344     0.984192
2     1.0   30.0  0.986497  0.985423  0.990265     0.986317
3     1.0   40.0  0.987976  0.987165  0.993334     0.987795
4     1.0   50.0  0.989496  0.988592  0.995114     0.989317
..    ...    ...       ...       ...       ...          ...
70    5.0  110.0  0.999018  0.997529  0.998769     0.998731
71    5.0  120.0  0.999177  0.997657  0.998451     0.998872
72    5.0  130.0  0.999315  0.997716  0.998228     0.998998
73    5.0  140.0  0.999219  0.997036  0.998470     0.998772
74    5.0  150.0  0.999425  0.997360  0.998458     0.999012

[75 rows x 6 columns]


In [ ]:
plt.scatter(test_label,nn_test_label,s=4)
plt.plot([0,40],[0,40],color = 'orange')
plt.xlabel('Test data')
plt.ylabel('NN prediction')
plt.title('NN prediction versus test set: 475-year RP with impeding factors')
plt.show()

In [ ]:
plt.hist(output_data, bins=40)
plt.title('Histogram: $T_{FR,50}$ training data')
plt.grid(alpha = 0.5)
plt.show()

In [ ]:
'''Perform sensitivity analysis'''
n_component = 41
n_increases = 21

t0 = time.time()

sens_matrix = np.zeros((n_component,n_increases))

''' Example discussed while teaching how to use the code'''

input_to_evaluate = np.ones((1,41))
input_to_evaluate[4] = 2.5
input_to_evaluate = scaler.transform(input_to_evaluate)
output_value = mlp.predict(input_to_evaluate)

for i in range(n_component):
        for j in range(n_increases):
                vector_input = np.ones((1,n_component))
                vector_input[0][i] = 1 + j*0.1
                vector_input = scaler.transform(vector_input)
                pred_val = mlp.predict(vector_input)
                sens_matrix[i][j] = pred_val[0]
t1 = time.time()

print(t1-t0)

with open('sensitivity_matrix.pkl','wb') as f:
        pickle.dump(sens_matrix,f)

np.savetxt('sensitivity_matrix.csv',sens_matrix,delimiter=',')
